In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth


In [2]:
from huggingface_hub import login

login(token="YOUR-TOKEN-HUGINGFACE")

In [3]:
from unsloth import FastLanguageModel
import torch

model1, tokenizer1 = FastLanguageModel.from_pretrained(
    model_name = "thailevann/cot_refine",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-09 07:23:14.516294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757402594.816829      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757402594.886679      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.2: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/264M [00:00<?, ?B/s]

Unsloth 2025.9.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
import torch
import torch.nn.functional as F
def generate_answer_cot(context, question, options):
    options_text = "\n".join([f"{i+1}. {opt}" for i, opt in enumerate(options)])


    problem = (
                    f"{context}\n\n"
                    f"Câu hỏi: {question}\n"
                    f"Lựa chọn:\n{options_text}\n\n"
                    f"Hãy suy luận theo **Chain-of-Thought (CoT)** để tìm ra lựa chọn đúng.\n"
                    f"Viết phần suy luận vào thẻ <think>.  Sau đó, liệt kê nhãn tương ứng của từng lựa chọn trong thẻ <labels>.\n"
        )



    messages = [{"role": "user", "content": problem}]
    text = tokenizer1.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=True,
    )
    # Tokenize & gen
    inputs = tokenizer1(text, return_tensors="pt").to("cuda")
    '''
    output_ids = model1.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        do_sample=False,
    )
    '''

    output = model1.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        do_sample=False,  # hoặc False tùy bạn
        output_scores=True,
        return_dict_in_generate=True,
    )

    
    # Lấy chuỗi token sinh ra
    sequences = output.sequences[0]   # batch=1
    decoded = tokenizer1.decode(sequences, skip_special_tokens=False)
    
    # Tính xác suất
    all_probs = []
    for i, score in enumerate(output.scores):
        # score: [batch, vocab_size]
        probs = F.softmax(score, dim=-1)  # xác suất
        token_id = sequences[inputs["input_ids"].shape[1] + i]  # token sinh ở step này
        token_prob = probs[0, token_id].item()
        all_probs.append((token_id, token_prob, tokenizer1.decode([token_id])))


    #decoded = tokenizer1.decode(output_ids[0], skip_special_tokens=True)
    return decoded, all_probs

In [7]:
def generate_answer_refine(context, question, options, reasoning):
    options_text = "\n".join([f"{i+1}. {opt}" for i, opt in enumerate(options)])


    problem = (
            f"{context}\n\n"
            f"Câu hỏi: {question}\n"
            f"Suy luận chi tiết cho từng lựa chọn:\n{reasoning.strip()}\n"
            f"Các lựa chọn:\n{options_text}\n\n"
            "Nhiệm vụ:\n"
            "1. Đọc phần suy luận của từng lựa chọn.\n"
            "2. Chỉ gán nhãn 1 cho phương án được tổng kết là hợp lý nhất.\n"
            "3. Tất cả các lựa chọn khác gán nhãn 0, ngay cả khi suy luận nói 'có thể hợp lý trong điều kiện đặc biệt'.\n"
            "4. Kết quả phải hoàn toàn nhất quán với phần tổng kết suy luận.\n"
            "5. Nếu dựa vào context và question mà không thể xác định option nào hợp lý nhất, gán 0 cho tất cả các option.\n"
            "6. Liệt kê nhãn tương ứng của từng lựa chọn theo đúng thứ tự, trong thẻ <labels>, ví dụ: <labels>1 0 0 0</labels>."
        )


    messages = [{"role": "user", "content": problem}]
    text = tokenizer1.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False,
    )
    # Tokenize & gen
    inputs = tokenizer1(text, return_tensors="pt").to("cuda")

    output = model1.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        do_sample=False,  # hoặc False tùy bạn
        output_scores=True,
        return_dict_in_generate=True,
    )

    
    # Lấy chuỗi token sinh ra
    sequences = output.sequences[0]   # batch=1
    decoded = tokenizer1.decode(sequences, skip_special_tokens=False)
    
    # Tính xác suất
    all_probs = []
    for i, score in enumerate(output.scores):
        # score: [batch, vocab_size]
        probs = F.softmax(score, dim=-1)  # xác suất
        token_id = sequences[inputs["input_ids"].shape[1] + i]  # token sinh ở step này
        token_prob = probs[0, token_id].item()
        all_probs.append((token_id, token_prob, tokenizer1.decode([token_id])))


    #decoded = tokenizer1.decode(output_ids[0], skip_special_tokens=True)
    return decoded, all_probs

In [12]:
def find_option_span(tokens_with_probs, option):
    """
    Trả về (start_idx, end_idx) trong tokens_with_probs nếu tìm thấy option,
    ngược lại trả về None.
    """
    token_texts = [t for _, _, t in tokens_with_probs]
    option_tokens = list(option)  # naive tách theo ký tự
    
    # Ghép lại rồi split để khớp "3 giờ"
    joined = "".join(token_texts)
    pos = joined.find(option)
    if pos == -1:
        return None
    
    # Map lại pos từ char -> token index
    char_pos = 0
    for i, tok in enumerate(token_texts):
        next_pos = char_pos + len(tok)
        if char_pos <= pos < next_pos:
            start_idx = i
        if char_pos < pos + len(option) <= next_pos:
            end_idx = i + 1
            return (start_idx, end_idx)
        char_pos = next_pos
    
    return None

In [14]:
def find_cutoff_index(tokens_with_probs, cutoff_phrase="Tổng kết"):
    """
    Tìm index token nơi xuất hiện cutoff_phrase ("Tổng kết").
    Nếu không thấy thì fallback về </think> hoặc <labels>.
    """
    token_texts = [t for _, _, t in tokens_with_probs]
    joined = "".join(token_texts)

    # Tìm theo string
    pos = joined.find(cutoff_phrase)
    if pos != -1:
        # Map từ char pos -> token index
        char_pos = 0
        for i, tok in enumerate(token_texts):
            next_pos = char_pos + len(tok)
            if char_pos <= pos < next_pos:
                return i
            char_pos = next_pos

    # fallback
    for i, tok in enumerate(token_texts):
        if "</think>" in tok or "<labels>" in tok:
            return i
    return len(tokens_with_probs)

In [5]:
def extract_option_probs(tokens_with_probs, options):
    texts = [tok for _, _, tok in tokens_with_probs]
    full_text = "".join(texts)

    # tìm vị trí <labels> cuối cùng
    start = full_text.lower().rfind("<labels>")
    end   = full_text.lower().rfind("</labels>")
    if start == -1 or end == -1 or end <= start:
        #print("[WARN] Không tìm thấy <labels> hoặc </labels>")
        return {}

    # map offset -> token index
    idx_map = []
    pos = 0
    for i, t in enumerate(texts):
        idx_map.append((pos, pos+len(t), i))
        pos += len(t)

    start_token = min(i for s, e, i in idx_map if s >= start)
    end_token   = max(i for s, e, i in idx_map if e <= end)
    segment = tokens_with_probs[start_token:end_token+1]

    labels = []
    for _, prob, tok in segment:
        norm_tok = tok.strip().lower().strip(",")
        if norm_tok in ("yes", "no"):
            labels.append((norm_tok, prob))

    #if len(labels) != len(options):
        #print(f"[WARN] Số label ({len(labels)}) != số option ({len(options)})")

    results = {}
    for opt, (lab, pr) in zip(options, labels):
        #print(f"[DEBUG] Option {opt} -> label={lab}, prob={pr:.4f}")
        if lab == "yes":
            results[opt] = (pr, 1-pr)
        else:
            results[opt] = (1-pr, pr)
    return results

In [6]:
import re

def extract_think_content(predicted_labels: str) -> str:
    """
    Trích xuất nội dung trong thẻ <think>...</think>
    """
    match = re.search(r"<think>(.*?)</think>", predicted_labels, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return ""

In [7]:
import math
import numpy as np

def softmax_binary(logits_yes, logits_no):
    """Softmax cho 2 nhãn yes/no"""
    exp_yes = math.exp(logits_yes)
    exp_no  = math.exp(logits_no)
    s = exp_yes + exp_no
    return exp_yes/s, exp_no/s

def ensemble_sweep_label(probs_cot, probs_refine, options, start=0.05, end=1.0, step=0.05, eps=1e-8):
    """
    Ensemble CoT + Refine với log + weighted sum + softmax nhị phân
    Trả về list: [(w_refine, final_pred, final_probs, labels)]
    """
    results = []
    weights = [round(start + step*i, 2) for i in range(int((end-start)/step)+1)]
    
    for w_refine in weights:
        w_cot = 1.0 - w_refine
        
        final_probs = {}
        labels = []
        
        # Duyệt từng option
        for opt in options:
            # Logits
            log_yes = math.log(probs_cot[opt][0]+eps)*w_cot + math.log(probs_refine[opt][0]+eps)*w_refine
            log_no  = math.log(probs_cot[opt][1]+eps)*w_cot + math.log(probs_refine[opt][1]+eps)*w_refine
            
            # Softmax nhị phân
            prob_yes, prob_no = softmax_binary(log_yes, log_no)
            final_probs[opt] = prob_yes  # chỉ lưu xác suất yes
            
            # Nhãn
            label = 'yes' if prob_yes > prob_no else 'no'
            labels.append(label)
        
        # final_pred: option có xác suất yes cao nhất
        final_pred = max(final_probs, key=final_probs.get)
        
        results.append((w_refine, final_pred, final_probs, labels))
    
    return results

In [8]:
'''
question = "Hoàn thành dự án cá nhân cần khoảng thời gian bao lâu?"
context = """
Tôi vừa hoàn thành dự án cá nhân mà mình đã bắt đầu từ lâu.
"""
options = [
"3 giờ",
"5 ngày",
"2 tháng",
"6 tháng"
]
answer_cot , prob_cot= generate_answer_cot(context, question, options)
old_reason_cot = extract_think_content(str(answer_cot)).split("<think>")[-1]
answer_refine , prob_refine = generate_answer_refine(context, question, options, old_reason_cot)

probs_cot = extract_option_probs(prob_cot, options)
probs_refine = extract_option_probs(prob_refine, options)

ensemble_sweep_label(probs_cot, probs_refine, options)
'''

In [5]:
from datasets import load_dataset

dataset = load_dataset("thailevann/durationQA_track8_vlsp2025")

In [17]:
import json
from tqdm import tqdm
import os
import time

output_dir = "results_by_weight"
os.makedirs(output_dir, exist_ok=True)

samples = dataset['test']

probs_cot_all = {}
probs_refine_all = {}

cot_file = os.path.join(output_dir, "results_cot.jsonl")

total_pred_time = 0.0

with open(cot_file, "w", encoding="utf-8") as f_cot:
    for sample in tqdm(samples, desc="Processing samples"):
        qid = sample['qid']
        context = sample['context']
        question = sample['question']
        options = sample['options']

        t0 = time.time()
        # --- CoT ---
        answer_cot, prob_cot = generate_answer_cot(context, question, options)
        probs_cot = extract_option_probs(prob_cot, options)
        probs_cot_all[qid] = probs_cot
        labels_cot = ['yes' if probs_cot[opt][0] > 0.5 else 'no' for opt in options]
        out_cot = {"qid": qid, "labels": labels_cot, "context": context, "question": question,"options": options}
        f_cot.write(json.dumps(out_cot, ensure_ascii=False) + "\n")

        # --- Refine ---
        old_reason_cot = extract_think_content(str(answer_cot)).split("<think>")[-1]
        answer_refine, prob_refine = generate_answer_refine(context, question, options, old_reason_cot)
        probs_refine = extract_option_probs(prob_refine, options)
        probs_refine_all[qid] = probs_refine

        # --- Ensemble sweep và ghi file ngay ---
        results = ensemble_sweep_label(probs_cot, probs_refine, options)
        for w_refine, pred, final_probs, labels in results:
            filename = os.path.join(output_dir, f"results_wrefine_{w_refine:.2f}.jsonl")
            with open(filename, "a", encoding="utf-8") as f:
                f.write(json.dumps({"qid": qid, "labels": labels, "context": context, "question": question,"options": options}, ensure_ascii=False) + "\n")

            # Nếu nhãn khác CoT thì in ra
            if labels != labels_cot:
                print(f"qid={qid} | CoT={labels_cot} -> Ensemble={labels} | w_refine={w_refine}")

        t1 = time.time()
        total_pred_time += t1 - t0

# --- Thời gian trung bình predict/sample ---
avg_time = total_pred_time / len(samples)
print(f"\nAverage prediction time per sample: {avg_time:.4f} seconds")